<a href="https://colab.research.google.com/github/MehakArora/ImageInpainting/blob/main/RPCA_Cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade opencv-contrib-python
!pip install tensorflow-gpu==2.0

In [2]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.python.client import device_lib

#print('[INFO]', tf.config.experimental.list_physical_devices('GPU')[0])

2.0.0


In [3]:
import os
import cv2
print(cv2.__version__)
import numpy as np

import matplotlib.pyplot as plt
#matplotlib inline
from skimage.metrics import structural_similarity as SSIM

from mpl_toolkits.axes_grid1 import ImageGrid

4.5.5


In [11]:
from zipfile import ZipFile as zip

from tensorflow import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

sample_images = x_train[:10000]
sample_labels = y_train[:10000]


import tensorflow as tf
def dice_coef(y_true, y_pred):
   y_true_f = keras.backend.flatten(y_true)
   y_pred_f = keras.backend.flatten(y_pred)
   intersection = keras.backend.sum(y_true_f * y_pred_f)
   return (2. * intersection) / (keras.backend.sum(y_true_f + y_pred_f))

def get_psnr(y_true, y_pred):
  val = tf.image.psnr(y_true, y_pred, 255)
  return val
  

def get_ssim(y_true, y_pred):
  y_true_tensor = tf.image.convert_image_dtype(y_true, tf.float64)
  y_pred_tensor = tf.image.convert_image_dtype(y_pred, tf.float64)
  val = tf.image.ssim(y_true_tensor, y_pred_tensor, 255)
  return float(val)


def createMask( img):
    ## Prepare masking matrix
    
    mask = np.full(img.shape, 255, np.uint8)
    for _ in range(np.random.randint(1, 10)):
      # Get random x locations to start line
      x1, x2 = np.random.randint(1, img.shape[0]), np.random.randint(1, img.shape[1])
      # Get random y locations to start line
      y1, y2 = np.random.randint(1, img.shape[0]), np.random.randint(1, img.shape[1])
      # Get random thickness of the line drawn
      thickness = np.random.randint(1, 3)
      # Draw black line on the white mask
      cv2.line(mask,(x1,y1),(x2,y2),(1,1,1),thickness)

    # Perforn bitwise and operation to mak the image
    masked_image = cv2.bitwise_and(img, mask)

    return masked_image, mask

#masked_image, mask = createMask(img)
def image_LS(image ,maskes_image, L_image, S_image):
    
    fig, ax = plt.subplots(1,4,figsize=(16,6))
    ax[0].imshow(image.astype('uint8'))
    ax[0].set_title("Original image", fontsize = 16)
    ax[1].imshow(masked_image.astype('uint8'))
    ax[1].set_title("masked image", fontsize = 16)
    ax[2].imshow(L_image.astype('uint8'))
    ax[2].set_title("Low rank matrix L", fontsize = 16)
    ax[3].imshow(np.abs(S_image).astype('uint8'))
    ax[3].set_title("Sparse matrix S", fontsize = 16);
    


def s_tau(X, tau):
        """Shrinkage operator
            Sτ [x] = sign(x) max(|x| − τ, 0)
        """
        return np.sign(X)*np.maximum(np.abs(X)-tau,0)
    
def d_tau(X, mu):
        """Singular value thresholding operator
            Dτ (X) = USτ(Σ)V∗, where X = UΣV∗
        """
        # singular value decomposition
        u, s, vh = np.linalg.svd(X, full_matrices=False)

        # Shrinkage of singular values
        tau = 1.0/mu
        s = s[s>tau] - tau
        rank = len(s)
        # reconstruct thresholded 2D array
        return  np.dot(u[:, 0:rank] * s, vh[0:rank,:]), rank


def RobustPCA(M, tol, max_iter):
    
        size = M.shape
        
        # initialize S and Y (Lagrange multiplier)
        S = np.zeros(size)
        Y = np.zeros(size)

        # if lamb and mu are not set, set with default values
        mu = np.prod(size)/4.0/np.sum(np.abs(M))
        lamb = 1/np.sqrt(np.max(size))

        # Alternating update
        for i in range(max_iter):
            L, rank = d_tau(M-S+1.0/mu*Y, mu)
            S = s_tau(M-L+1.0/mu*Y, lamb/mu)

            # Calculate residuals
            residuals = M-1.2*L-S
            residuals_sum = np.linalg.norm(residuals,'fro')

            # Check convergency
            if residuals_sum <= 0.0001 * np.linalg.norm(M,'fro'):
                break

            Y = Y + mu*residuals

        # Check if the fit is converged
        if residuals_sum > 0.0001* np.linalg.norm(M,'fro'):
            print('Not converged!')
            print('Total error: %f, allowed tolerance: %f'%(residuals_sum, tol))
        else:
            print('Converged!')

        return L, S, rank

def PCA(M, max_iter):
    
        size = M.shape
        
        # initialize S and Y (Lagrange multiplier)
        S = np.zeros(size)
        Y = np.zeros(size)

        # if lamb and mu are not set, set with default values
        mu = np.prod(size)/4.0/np.sum(np.abs(M))
        lamb = 1/np.sqrt(np.max(size))

        # Alternating update
        for i in range(max_iter):
            L, rank = d_tau(M-S+1.0/mu*Y, mu)
            S = s_tau(M-L+1.0/mu*Y, lamb/mu)

            # Calculate residuals
            residuals = M-L-S
            residuals_sum = np.sum(np.abs(residuals))

            Y = Y + mu*residuals
        return L, S, rank

def dice_coef(y_true, y_pred):
    y_true = (y_true).astype('float64')
    y_pred = (y_pred).astype('float64')
    y_true_f = keras.backend.flatten(y_true)
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (keras.backend.sum(y_true_f + y_pred_f))

def get_psnr(y_true, y_pred):
  val = tf.image.psnr(y_true, y_pred, 255)
  return val
  

def get_ssim(y_true, y_pred):
  y_true_tensor = tf.image.convert_image_dtype(y_true, tf.float32)
  y_pred_tensor = tf.image.convert_image_dtype(y_pred, tf.float32)
  val = tf.image.ssim(y_true_tensor, y_pred_tensor, 255)
  return float(val)

p_c = np.zeros(9999)
s_c = np.zeros(9999)
d_c = np.zeros(9999)

for j in range (9999):
    img = sample_images[j]
    masked_image, mask = createMask(img)
    image = masked_image/255
    #print("Robust PCA ...");
    print ("for image",j+1)
    

    L_image = image*0
    S_image = image*0
    rank = [];

    for i in range(3):
        L_image[:,:,i] ,S_image[:,:,i], rnk = RobustPCA(image[:,:,i], tol = 0.5, max_iter=100000)
        rank.append(rnk)

    outputRPCA = (L_image*255).astype('uint8')
    p_c[j] = get_psnr(img,L_image)
    s_c[j] = get_ssim(img,L_image)
    d_c[j] = dice_coef(img,L_image)

    #outputRPCA = (L_image*255).astype('uint8')
    #image_LS(image*255, L_image*255, S_image*255, thres=0)
    #image_LS(image*255, outputRPCA, S_image*255, thres=0)
    #image_LS(img, image*255, outputRPCA, S_image*255)













Streaming output truncated to the last 5000 lines.
for image 8750
Converged!
Converged!
Converged!
for image 8751
Converged!
Converged!
Converged!
for image 8752
Converged!
Converged!
Converged!
for image 8753
Converged!
Converged!
Converged!
for image 8754
Converged!
Converged!
Converged!
for image 8755
Converged!
Converged!
Converged!
for image 8756
Converged!
Converged!
Converged!
for image 8757
Converged!
Converged!
Converged!
for image 8758
Converged!
Converged!
Converged!
for image 8759
Converged!
Converged!
Converged!
for image 8760
Converged!
Converged!
Converged!
for image 8761
Converged!
Converged!
Converged!
for image 8762
Converged!
Converged!
Converged!
for image 8763
Converged!
Converged!
Converged!
for image 8764
Converged!
Converged!
Converged!
for image 8765
Converged!
Converged!
Converged!
for image 8766
Converged!
Converged!
Converged!
for image 8767
Converged!
Converged!
Converged!
for image 8768
Converged!
Converged!
Converged!
for image 8769
Converged!
Converged!


In [17]:
np.average(p_c)
np.average(s_c)
np.average(d_c)

0.8009232586571943